In [ ]:
%load_ext sql

In [ ]:
import time
import configparser
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Read configuration file and connect to database
from etl import create_client
from sql_queries import analytics_tables, staging_tables
import configparser

config = configparser.ConfigParser()
config.read('dwh.cfg')
KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")
DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
redshift_client = create_client(KEY, SECRET, 'redshift')
myClusterProps = redshift_client.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']

conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
%sql $conn_string
%sql SET search_path TO dist;

### 1. Check if entries were created in each table

In [ ]:
# Check if each table has enrtries
entries = []

for t in analytics_tables:
    result = %sql select count(*) from $t
    entries.append(result[0][0])

entries_df = pd.DataFrame({'entries': entries}, index=analytics_tables)
entries_df.plot.bar()
entries = []

for t in staging_tables:
    result = %sql select count(*) from $t
    entries.append(result[0][0])

entries_df = pd.DataFrame({'entries': entries}, index=staging_tables)
entries_df.plot.bar()

### 2. Check for duplicate songs from the same artists in staging_songs

In [ ]:
# Check for duplicate titles from an artist
%sql select artist_name, title, count(title) from staging_songs group by artist_name, title having count(*) > 1 order by count(title)

### 3. Check for missing artist locations

In [ ]:
# Check of missing data in songs files
all_artists = %sql select count(*) from artists
missing_locations = %sql select count(*) from staging_songs where artist_location is null
all_artists = all_artists[0][0]
missing_locations = missing_locations[0][0]
result_df = pd.DataFrame({'entries': [all_artists, missing_locations]}, index=['all artists', 'artists with missing locations'])
result_df.plot.barh()

#### 4. Check for NextSong actions that cannot be matched to songs

In [ ]:
# Check for missing data in NextSong actions that prevent it from being matched fully
all_events = %sql select count(*) from staging_events
all_events = all_events[0][0]
%sql create temp table next_songs as select * from staging_events where page like 'NextSong'
next_song_actions = %sql select count(*) from next_songs
next_song_actions = next_song_actions[0][0]
title_match = %sql select count(*) from next_songs inner join staging_songs on staging_songs.title like next_songs.song
title_match = title_match[0][0]

results = pd.DataFrame({'entries': [all_events, next_song_actions, title_match]}, index=['All events', 'All NextSong actions', 'NextSong Actions with matching song title'])
results.plot.bar()

### 5. Check for runtimes of each ETL step

In [ ]:
# Check SQL runtimes for loading and transforming
%sql SET search_path TO dist;
rt_df = %sql select * from dashboard
rt_df = rt_df.DataFrame()
rt_df.plot(kind='bar', title='Runtimes of ETL steps in seconds', legend=True, label=rt_df.step.values.tolist())

### 6. Check the Top 10 songs in table songplays

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

top10_df = %sql select songs.title, count(*) as played from songplays join songs on songs.song_id like songplays.song_id join artists on artists.artist_id like songs.artist_id group by songs.title order by played desc limit 10
top10_df = top10_df.DataFrame()
top10_df = top10_df.set_index('title')
top10_df = top10_df.sort_values(by='played')
top10_df.plot.barh(sort_columns=False)